# Predict Tomorrows Bitcoin Close Price using Classical Machine Learning

In [ ]:
from main import run_prediction_pipeline
from helpers.llm_support import get_prompt

print("Forecasting Tomorrows Bitcoin Close Price...")
MODELS_DIRECTORY = r'C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\models\20251112_170018'
df, predictions = run_prediction_pipeline(models_dir = MODELS_DIRECTORY)
prompt = get_prompt(predictions, df['close'].iloc[-1])
print(predictions)

c:\Users\baile\anaconda3\envs\crypto_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Forecasting Tomorrows Bitcoin Close Price...
--- Starting Data Preprocessing with Temporal Alignment ---
--- Data Preprocessing Complete ---
--- Starting Prediction ---
--- Starting Prediction Pipeline ---
Loading artifacts from: C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\models\20251112_170018
--- Performing Feature Engineering ---

Forecast for the Next Day:
  - Linear Regression: 101669.06
  - Ridge Regression: 101822.13
  - Random Forest: 101228.99
  - XGBoost: 98411.44
{'Linear Regression': 101669.05650693519, 'Ridge Regression': 101822.13013941789, 'Random Forest': 101228.99116666701, 'XGBoost': 98411.44}


## Feed the Classical Machine Learning Predictions into the LLM to have it search the web with context and make a prediction for Tomorrow UTC Close (1600 prior day for PST).

In [ ]:
from google import genai
from google.genai import types
from helpers.queries import urls
from helpers.llm_support import add_citations
from dotenv import load_dotenv #comment this out
import os

# Load .env file
load_dotenv()
api_key = os.getenv("gemini_api_key") #replace with your api key
client = genai.Client()

# Build the tools list with each URL as url_context
tools = [
  {"url_context": {}}, #specific user defined urls
  types.Tool(google_search=types.GoogleSearch()) #dynamic search
]

urls = urls() #pulls from dependency

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[prompt + "Look through these and some of your own URLs not listed here: {urls}"],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=-1), #dynamically decides how much it needs to think
        temperature=0.0, # "Highest probability" response. 
        tools=tools
    )
)

text_with_citations = add_citations(response)
print(text_with_citations)